In [1]:
import chromadb
# tunning chroma db with custom setting
from chromadb.config import Settings

In [2]:
import pandas as pd
df = pd.read_csv("medium_post_titles.csv")
df = df.dropna()
df = df[-df["subtitle_truncated_flag"]]
topics_of_interest = ["artificial-intelligence", "machine-learning", "data-science"]
df=df[df["category"].isin(topics_of_interest)]
df["text"]=df["title"]+df["subtitle"]
df["meta"] =df.apply( lambda x: {
    "text": x["text"],
    "category": x["category"]
}, axis=1)

In [3]:
# chroma_client = chromadb.Client(Settings(
#     persist_directory="medium-chroma-db", #place where db will be saved in current working directory
#     chroma_db_impl="duckdb+parquet" #default value
# ))
chroma_client = chromadb.PersistentClient(path="medium-chroma-db") # poza pamiecia na dysku # persistent memory

article_collection = chroma_client.create_collection(name="medium_article")

In [4]:
#inserting data
article_collection.upsert(
    ids=[f"{x}" for x in df.index.tolist()],
    documents=df["text"].tolist(),
    metadatas = df["meta"].tolist()
)

In [5]:
qry_str =  "best data science library?"
article_collection.query(query_texts=qry_str, n_results=2) 

{'ids': [['65427', '2586']],
 'distances': [[0.6087137460708618, 0.6778908967971802]],
 'metadatas': [[{'category': 'data-science',
    'text': 'My Favorite Data Science/Machine Learning ResourcesA summary of sources to get into Data Science'},
   {'category': 'artificial-intelligence',
    'text': '5 Resources to Inspire Your Next Data Science ProjectDon’t worry — getting started is the hardest part'}]],
 'embeddings': None,
 'documents': [['My Favorite Data Science/Machine Learning ResourcesA summary of sources to get into Data Science',
   '5 Resources to Inspire Your Next Data Science ProjectDon’t worry — getting started is the hardest part']],
 'uris': None,
 'data': None}